In [44]:
import json
import numpy as np

In [131]:
friend_data = np.genfromtxt('grupee_data/friends.csv', delimiter=',', dtype=int)
friendships = [[] for _ in range(np.max(friend_data[:,0])+1)]

for id in set(friend_data[:,0]):
    friendships[id] = [id, friend_data[friend_data[:, 0] == id, 1]]

with open('grupee_data/preferences.json') as file:
    preferences = json.load(file)

with open('grupee_data/n_concerts.txt') as file:
    n_concerts = file.read().split('\n')[1:-1]
    concerts = np.array([(x.split(':')[0], x.split(':')[1]) for x in n_concerts])
    file.close()

[10, array([321])]
